In [1]:
library(data.table)
library(stringr)
library(pheatmap)
library(ape)
library(ggplot2)
library(Rtsne)
library(sva)
library(VennDiagram)
library(readxl)
library(energy)
library(MASS)
library(quantreg)
library(limma)
library(Glimma)
library(edgeR)
library(RColorBrewer)
library(minerva)
library(dbscan)
library(amap)
library(NbClust)  
library(apeglm)
library(ggrepel)

Loading required package: mgcv
Loading required package: nlme
This is mgcv 1.8-31. For overview type 'help("mgcv-package")'.
Loading required package: genefilter
Loading required package: BiocParallel
Loading required package: grid
Loading required package: futile.logger

Attaching package: ‘futile.logger’

The following object is masked from ‘package:mgcv’:

    scat


Attaching package: ‘VennDiagram’

The following object is masked from ‘package:ape’:

    rotate


Attaching package: ‘MASS’

The following object is masked from ‘package:genefilter’:

    area

Loading required package: SparseM

Attaching package: ‘SparseM’

The following object is masked from ‘package:base’:

    backsolve



In [2]:
setwd('~/ownCloud/Meng/')

In [3]:
path <- "100_plus_project/100plus_data_Frank_2019-07-09/peptide data.tsv"
raw_data <- read.table(path, header=T, stringsAsFactors=F, sep = "\t")

dim(raw_data)

[1] 28208   476

In [4]:
peptide_quality <- raw_data[,c(1, 2, grep("qvalue_", colnames(raw_data)))]
peptide_intensity <- raw_data[,c(1, 2, grep("intensity_", colnames(raw_data)))]

In [8]:
peptide_quality

peptide_id,protein,qvalue_20190325_121_S16_020_CHC_011,qvalue_20190325_125_S16_044_CHC_007,qvalue_20190325_127_S16_094_CHC_004,qvalue_20190325_133_2017_006_CHC_001,qvalue_20190325_16_04_037_AD_008,qvalue_20190325_183_S10_292_AD_010,qvalue_20190325_194_S02_055_ND_005,qvalue_20190325_31_01_184_AD_006,⋯,qvalue_20190606_141_2017_028_CHC_RERUN,qvalue_20190606_20_S00_166_AD_RERUN,qvalue_20190606_2_S05_265_AD_RERUN,qvalue_20190606_78_S14_029_ND_RERUN,qvalue_20190606_81_S09_128_ND_RERUN,qvalue_20190606_84_S12_088_ND_RERUN,qvalue_20190606_85_S09_244_ND_RERUN,qvalue_20190606_86_S12_086_ND_RERUN,qvalue_20190606_98_S08_153_ND_RERUN,qvalue_20190606_9_S93_025_AD_RERUN
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
_AAAAAAAAAAAAAAAGAGAGAK_.3,SLC12A2,3.144607e-07,7.056776e-20,2.407190e-05,5.242789e-03,7.778710e-12,4.678175e-05,9.896840e-27,2.458094e-11,⋯,2.343930e-32,5.039982e-13,2.716734e-06,8.272101e-07,3.789649e-17,2.230381e-05,4.466585e-09,1.873847e-11,1.755449e-14,1.528784e-18
_AAAAAAAAAAAPPAPPEGASPGDSAR_.3,CASKIN1,1.370624e-08,2.953944e-37,5.807807e-24,2.235007e-46,5.773969e-23,4.678175e-05,1.235743e-27,4.865253e-16,⋯,7.692545e-49,4.094094e-19,5.993776e-14,2.590186e-09,8.359839e-37,3.880940e-06,2.758219e-16,2.645381e-18,1.335255e-26,4.247500e-33
_AAAAAAAAAAAPPAPPEGASPGDSAR_.4,CASKIN1,2.084066e-06,7.599246e-28,4.298010e-23,3.852727e-08,1.496074e-11,2.159272e-04,1.069702e-19,5.637271e-03,⋯,1.924114e-30,7.588227e-09,2.413678e-02,1.929624e-05,8.486507e-10,7.426561e-05,6.963947e-04,1.595333e-13,3.990303e-18,1.715446e-15
_AAAAAAAAVPSAGPAGPAPTSAAGR_.3,IRS2,9.939043e-04,4.547814e-03,1.942234e-01,1.630938e-02,3.041983e-03,5.490039e-04,4.156453e-02,6.614350e-03,⋯,3.020637e-02,2.856976e-02,1.110777e-01,2.163313e-01,1.072073e-01,2.441134e-01,1.721514e-01,1.238253e-02,9.961095e-02,3.315855e-02
_AAADLM[Oxidation (M)]AYC[Carbamidomethyl (C)]EAHAK_.3,GNG2,9.381378e-02,6.353922e-02,1.204311e-01,9.639277e-02,8.976155e-02,1.312902e-01,3.973888e-02,5.582974e-02,⋯,1.802354e-01,3.583682e-02,7.174542e-02,1.685809e-01,5.956869e-02,1.755663e-01,9.075761e-02,2.201380e-02,1.438305e-02,1.573813e-01
_AAAEAALAAVLALEAGLSAEQR_.3,CARNS1,1.142591e-02,8.894188e-04,5.710995e-05,9.003613e-19,1.030242e-02,7.679407e-05,8.340500e-05,6.713185e-08,⋯,1.002523e-04,7.605606e-04,6.228133e-03,8.931441e-03,8.452090e-04,8.552809e-03,1.778420e-02,4.801719e-04,1.091590e-03,1.006080e-04
_AAAEVNQDYGLDPK_.2,FH,1.470804e-08,1.822464e-29,6.719825e-30,2.570764e-41,4.417423e-24,4.678175e-05,6.111389e-25,1.224173e-23,⋯,4.311060e-44,2.346692e-18,4.104011e-13,2.590503e-09,2.730655e-54,4.011691e-06,4.702812e-12,1.839183e-14,4.443803e-35,1.085544e-25
_AAAEVNQDYGLDPK_.3,FH,9.944966e-03,3.481280e-07,2.094297e-04,5.413839e-04,3.751567e-02,5.141489e-03,4.532487e-03,1.024421e-03,⋯,9.319224e-02,4.256307e-04,1.078838e-01,3.987017e-02,2.464572e-01,1.199101e-01,1.577585e-01,1.295709e-01,2.687584e-02,4.787238e-02
_AAAGELQEDSGLC[Carbamidomethyl (C)]VLAR_.3,EFHD2,2.902683e-05,3.307188e-03,5.412404e-06,1.254490e-04,1.176404e-09,4.780150e-05,6.637525e-02,3.466194e-04,⋯,5.390630e-24,2.123380e-05,1.958901e-01,6.179655e-02,5.058557e-02,7.778848e-02,2.003726e-02,1.103089e-02,2.247161e-04,1.410931e-13


In [69]:
metadata <- read_excel("100_plus_project/Metadata/2019_07_23_Proteomics_Metadata_forMeng_v3.xlsx",
                       sheet=1,col_names=TRUE, na="NA")
colnames(metadata)

New names:
• `autopsy` -> `autopsy...3`
• `GROUP` -> `GROUP...4`
• `Snumber` -> `Snumber...8`
• `Snumber` -> `Snumber...16`
• `GROUP` -> `GROUP...17`
• `tissuecode` -> `tissuecode...19`
• `autopsy` -> `autopsy...22`
• `tissuecode` -> `tissuecode...35`
• `` -> `...51`

[1] "nbb with colour code"                "nbb"                                
 [3] "autopsy...3"                         "GROUP...4"                          
 [5] "ABG"                                 "sampleprepRandomization (4 batches)"
 [7] "IHC order"                           "Snumber...8"                        
 [9] "zmRandomization (10 batches)"        "Mass-Spec filenames (1)"            
[11] "Mass-Spec filenames (2)"             "IHC nr"                             
[13] "Autopsy nr"                          "Diagnosis"                          
[15] "Date LCM"                            "Snumber...16"                       
[17] "GROUP...17"                          "IHC numbering"                      
[19] "tissuecode...19"                     "LCM slides"                         
[21] "LCM finished"                        "autopsy...22"                       
[23] "sex"                                 "age"                                
[25] "braak"                               "amyloid"                            
[27] "braaklb"                             "pmd"                                
[29] "ph"                                  "weight"                             
[31] "csf"                                 "apoe"                               
[33] "barcode"                             "id"                                 
[35] "tissuecode...35"                     "iduit"                              
[37] "datumuit"                            "uitvraag"                           
[39] "recipient"                           "clin. diagn."                       
[41] "dcode"                               "diagnosis"                          
[43] "dcodewk"                             "wcode"                              
[45] "region"                              "specific"                           
[47] "ocode"                               "storage"                            
[49] "freezer"                             "Clincal info"                       
[51] "...51"

In [70]:
names_sample <- as.array(colnames(peptide_quality)[3:ncol(peptide_quality)])
names_sample <- as.vector(apply(names_sample, 1, function(names){return (str_sub(names, 8))}))
all_filenames <- names_sample

filenames_map_index <- integer(length(names_sample))
for (i in 1:length(names_sample)){
    filenames_map_index[i] <- ifelse(length(which(metadata[,"Mass-Spec filenames (1)"] == names_sample[i]))==0,
                                     which(metadata[,"Mass-Spec filenames (2)"] == names_sample[i]), 
                                     which(metadata[,"Mass-Spec filenames (1)"] == names_sample[i]))
}

metadata <- metadata[filenames_map_index, ][c('nbb', 'Mass-Spec filenames (1)', 'Mass-Spec filenames (2)')]
metadata$inten_name <- all_filenames

In [71]:
metadata[metadata$nbb == "2008-054", ]

nbb,Mass-Spec filenames (1),Mass-Spec filenames (2),inten_name
<chr>,<chr>,<chr>,<chr>
2008-054,20190416_98_S08_153_ND_101,NA,20190416_98_S08_153_ND_101
2008-054,20190606_98_S08_153_ND_RERUN,20190416_98_S08_153_ND_101_GOOD,20190416_98_S08_153_ND_101_GOOD
2008-054,20190606_98_S08_153_ND_RERUN,20190416_98_S08_153_ND_101_GOOD,20190606_98_S08_153_ND_RERUN


In [27]:
nbb_count <- as.data.frame(table(metadata$nbb))
duplicate_nbb <- as.character(nbb_count[which(nbb_count$Freq > 1),]$Var1)

In [28]:
table(metadata[metadata$nbb %in% duplicate_nbb, ]$nbb)


1993-014 2000-076 2001-129 2001-141 2002-079 2003-057 2004-023 2004-030 
       2        2        2        2        2        2        2        2 
2005-071 2006-079 2007-027 2007-082 2008-054 2009-039 2009-075 2011-039 
       2        2        2        2        3        2        2        2 
2012-005 2012-049 2012-086 2012-088 2014-029 2016-107 2016-139 2017-027 
       2        2        2        2        2        2        2        2 
2017-028      777 
       2        2 

In [29]:
duplicate_nbb

[1] "1993-014" "2000-076" "2001-129" "2001-141" "2002-079" "2003-057"
 [7] "2004-023" "2004-030" "2005-071" "2006-079" "2007-027" "2007-082"
[13] "2008-054" "2009-039" "2009-075" "2011-039" "2012-005" "2012-049"
[19] "2012-086" "2012-088" "2014-029" "2016-107" "2016-139" "2017-027"
[25] "2017-028" "777"

In [46]:
duplicate_nbb_sample_index <- lapply(duplicate_nbb, function(nbb){return (which(metadata$nbb == nbb))})

In [47]:
the_sone <- duplicate_nbb_sample_index[[13]][1]

In [48]:
duplicate_nbb_sample_index[[13]] <- duplicate_nbb_sample_index[[13]][-1]

In [49]:
duplicate_nbb_sample_index <- as.data.frame(duplicate_nbb_sample_index)

In [50]:
duplicate_nbb_sample_index

c.60L..237L.,c.166L..229L.,c.22L..158L.,c.80L..143L.,c.140L..159L.,c.33L..89L.,c.17L..113L.,c.16L..98L.,c.194L..230L.,c.51L..53L.,⋯,c.44L..149L.,c.45L..222L.,c.204L..235L.,c.90L..233L.,c.49L..231L.,c.37L..225L.,c.68L..226L.,c.52L..227L.,c.63L..228L.,c.50L..224L.
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
60,166,22,80,140,33,17,16,194,51,⋯,44,45,204,90,49,37,68,52,63,50
237,229,158,143,159,89,113,98,230,53,⋯,149,222,235,233,231,225,226,227,228,224


In [42]:
exclude_sample_index <- numeric(0)
for(i in 1:ncol(duplicate_nbb_sample_index)){
    if(t.test(peptide_quality[, duplicate_nbb_sample_index[,i][1]+2], alternative = "greater",
              peptide_quality[, duplicate_nbb_sample_index[,i][2]+2], paired = TRUE)$p.value < 0.05){
        
        print(t.test(peptide_quality[, duplicate_nbb_sample_index[,i][1]+2], alternative = "greater",
                     peptide_quality[, duplicate_nbb_sample_index[,i][2]+2], paired = TRUE)$p.value)
        print(colnames(peptide_quality)[duplicate_nbb_sample_index[,i][1]+2])
        print(colnames(peptide_quality)[duplicate_nbb_sample_index[,i][2]+2])
        
        exclude_sample_index <- c(exclude_sample_index, duplicate_nbb_sample_index[,i][1])
    }
    else{
        
        print(t.test(peptide_quality[, duplicate_nbb_sample_index[,i][1]+2], alternative = "greater",
                     peptide_quality[, duplicate_nbb_sample_index[,i][2]+2], paired = TRUE)$p.value)
        print(colnames(peptide_quality)[duplicate_nbb_sample_index[,i][1]+2])
        print(colnames(peptide_quality)[duplicate_nbb_sample_index[,i][2]+2])
        
        exclude_sample_index <- c(exclude_sample_index, duplicate_nbb_sample_index[,i][2])
    }
}

[1] 3.193863e-134
[1] "qvalue_20190401_9_S93_025_AD_057"
[1] "qvalue_20190606_9_S93_025_AD_RERUN"
[1] 0
[1] "qvalue_20190424_20_S00_166_AD_177"
[1] "qvalue_20190606_20_S00_166_AD_RERUN"
[1] 1.07427e-115
[1] "qvalue_20190326_28_S01_297_AD_018"
[1] "qvalue_20190424_169_S01_297_AD_165"
[1] 1
[1] "qvalue_20190412_170_S01_328_AD_082"
[1] "qvalue_20190419_44_01_328_AD_126"
[1] 9.307299e-12
[1] "qvalue_20190419_40_S02_231_AD_124"
[1] "qvalue_20190424_173_S02_231_AD_159"
[1] 1
[1] "qvalue_20190328_176_S03_150_AD_033"
[1] "qvalue_20190412_7_S03_150_AD_079"
[1] 3.807267e-10
[1] "qvalue_20190326_177_S04_062_AD_023"
[1] "qvalue_20190416_6_04_062_AD_111"
[1] 0.9999998
[1] "qvalue_20190326_176_S04_088_AD_021"
[1] "qvalue_20190415_64_04_088_AD_94"
[1] 3.406229e-18
[1] "qvalue_20190502_2_S05_265_AD_194"
[1] "qvalue_20190606_2_S05_265_AD_RERUN"
[1] 1.32655e-259
[1] "qvalue_20190401_11_S06_264_AD_055"
[1] "qvalue_20190401_179_S06_264_AD_059"
[1] 2.577169e-77
[1] "qvalue_20190419_41_S07_116_AD_143"
[1] "

In [52]:
exclude_file_names <- metadata[c(exclude_sample_index, the_sone), ]$inten_name

In [54]:
exclude_file_names <- as.vector(sapply(exclude_file_names, function(names){return (str_sub(names, 11))}))

In [56]:
length(exclude_file_names)

[1] 27

### Check with what we have, if we have trouble

In [66]:
load(file = "100_plus_project/Metadata/metadata_clean_update20220901.Rdata")

In [68]:
sum(metadata$filenames %in% exclude_file_names)

[1] 0

In [ ]:
### Check with what we have, if we have trouble, no trouble

In [77]:
metadata[!(metadata$inten_name %in% exclude_file_names), ]

nbb,Mass-Spec filenames (1),Mass-Spec filenames (2),inten_name
<chr>,<chr>,<chr>,<chr>
2016-020,20190325_121_S16_020_CHC_011,NA,20190325_121_S16_020_CHC_011
2016-044,20190325_125_S16_044_CHC_007,NA,20190325_125_S16_044_CHC_007
2016-094,20190325_127_S16_094_CHC_004,NA,20190325_127_S16_094_CHC_004
2017-006,20190325_133_2017_006_CHC_001,NA,20190325_133_2017_006_CHC_001
2004-011,20190325_16_04_037_AD_008,NA,20190325_16_04_037_AD_008
2010-101,20190325_183_S10_292_AD_010,NA,20190325_183_S10_292_AD_010
2002-024,20190325_194_S02_055_ND_005,NA,20190325_194_S02_055_ND_005
2001-081,20190325_31_01_184_AD_006,NA,20190325_31_01_184_AD_006
1997-091,20190325_57_S97_235_AD_003,NA,20190325_57_S97_235_AD_003


In [81]:
save(all_filenames, file = "100_plus_project/Metadata/all_filenames_20220901.Rdata")

In [78]:
save(exclude_file_names, file = "100_plus_project/Metadata/exclude_file_names_20220901.Rdata")